# Agents

## ReAct Agent

### Simple Example Using Rlly Simple Tools

In [1]:
from kruppe.llm import OpenAILLM
from kruppe.algorithm.agents import ReActResearcher
from kruppe.functional.base_tool import BaseTool



In [ ]:
class ExampleReActResearcher(ReActResearcher):

    def _react_system_prompt(self) -> str:
        return """\
# Role
You answer questions through iterative cycles of reasoning and acting.

# Instruction
You answer questions by first thinking about the question, then call on tools retrieve information. Afterwards, you think about the retrieved information, and continue this process iteratively. When you have found an answer, you finish the task by generating FINISH[answer]. Unless it is the final FINISH action, always call a tool after each thought. Always respond with a thought.

# Output Format
- Always respond with an action at the end, and call on a tool.
- Only respond with one new action at a time.

# Example

## Example 1

### User
What is the capital of France?

### Assistant Response 1
#### Message
"Thought 1: I need to find the capital of France.
Action 1: I will call the get_capital tool to find the capital of France."

#### Tool Calls
get_capital(country="France)

### Assistant Response 2
#### Message
"Observation 1: Paris
Thought 2: The tool call returned Paris. So, the capital of France is paris.
Action 2: FINISH[Paris]"
"""

    def _react_user_prompt(self, query: str) -> str:
        return query

In [3]:
from kruppe.models import Document


class CountryHub(BaseTool):
    def get_capital(self, country: str) -> str:
        capitals = {
            "France": "Paris",
            "Germany": "Berlin",
            "Spain": "Madrid",
            "China": "Beijing",
        }
        return capitals.get(country, "Unknown"), [Document(text="Trust me", metadata={})]

    def get_continent(self, country: str) -> str:
        continents = {
            "France": "Europe",
            "Germany": "Europe",
            "Spain": "Europe",
            "China": "Asia",
        }
        return continents.get(country, "Unknown"), [Document(text="Trust me", metadata={})]

    def get_capital_schema(self):
        return {
            "type": "function",
            "function": {
                "name": "get_capital",
                "description": "Get the capital of a country",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "country": {"type": "string", "description": "The name of the country"},
                    },
                    "required": ["country"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    
    def get_continent_schema(self):
        return {
            "type": "function",
            "function": {
                "name": "get_continent",
                "description": "Get the continent of a country",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "country": {"type": "string", "description": "The name of the country"},
                    },
                    "required": ["country"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }



In [4]:
toolhub1 = CountryHub()

researcher = ExampleReActResearcher(
    llm=OpenAILLM(model="gpt-4.1"),
    toolkit=[
        toolhub1.get_capital,
        toolhub1.get_continent
    ]
)

In [5]:
query = "What is the capital and continent of China?"
researcher._messages = [
    {"role": "system", "content": researcher._react_system_prompt()},
    {"role": "user", "content": researcher._react_user_prompt(query)}
]

reason_messages, action = await researcher.reason(1)
print(reason_messages)
print(action)

[{'role': 'assistant', 'content': 'Thought 1: I need to find both the capital city of China and the continent on which China is located.\nAction 1: get_capital(country="China")\n'}]
get_capital(country="China")


In [6]:
tool_messages, obs, sources = await researcher.act(action)
print(tool_messages)
print(obs)
print(sources)

[{'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_5puJ6XcvJkyDpf0exwl9oyw5', 'type': 'function', 'function': {'name': 'get_capital', 'arguments': '{"country": "China"}'}}]}, {'role': 'tool', 'tool_call_id': 'call_5puJ6XcvJkyDpf0exwl9oyw5', 'content': 'Observation get_capital(country="China"): Beijing\n'}]
Observation get_capital(country="China"): Beijing

[Document(text='Trust me', id=UUID('138e632d-2b98-47a4-8737-3c0c8b5349e2'), metadata={'publication_time': ''})]


In [7]:
await researcher.execute(query)

Thought 1: I need to find both the capital city and the continent in which China is located.
Action 1: get_capital(country="China")
Observation 1: Beijing
Thought 2: The capital of China is Beijing. Now I need to find out which continent China is located in.
Action 2: get_continent(country="China")
Observation 2: Asia
Thought 3: I now know that the capital of China is Beijing and the continent is Asia.
Action 3: FINISH[The capital of China is Beijing and it is located on the continent of Asia.]


Response(text='the capital of china is beijing and it is located on the continent of asia.', sources=[Document(text='Trust me', id=UUID('59c7088a-4fc0-4e3e-beda-98f59209d385'), metadata={'publication_time': ''}), Document(text='Trust me', id=UUID('237f2de2-c606-4545-8b24-3df486f2c4c8'), metadata={'publication_time': ''})])

## Librarian Agent

In [1]:
from kruppe.llm import OpenAILLM, OpenAIEmbeddingModel
from kruppe.functional.docstore.mongo_store import MongoDBStore
from kruppe.functional.rag.vectorstore.chroma import ChromaVectorStore
from kruppe.functional.rag.index.vectorstore_index import VectorStoreIndex
from kruppe.functional.rag.retriever.simple_retriever import SimpleRetriever

reset_db=True


db_name = "kruppe_librarian"
collection_name = "playground_3"

# Create doc store
unique_indices = [['title', 'datasource']] # NOTE: this is important to avoid duplicates
docstore = await MongoDBStore.acreate_db(
    db_name=db_name,
    collection_name="playground",
    unique_indices=unique_indices,
    reset_db=reset_db
)

# Create vectorstore index
embedding_model = OpenAIEmbeddingModel()
vectorstore = ChromaVectorStore(
    embedding_model=embedding_model,
    collection_name=collection_name,
    persist_path='/Volumes/Lexar/Daniel Liu/vectorstores/kruppe_librarian'
)

if reset_db:
    vectorstore.clear()
    
index = VectorStoreIndex(vectorstore=vectorstore)

retriever = SimpleRetriever(index=index)

In [2]:
from kruppe.functional.ragquery import RagQuery
from kruppe.functional.llmquery import LLMQuery
from kruppe.functional.newshub import NewsHub
from kruppe.functional.finhub import FinHub

from kruppe.data_source.news.nyt import NewYorkTimesData
from kruppe.data_source.news.ft import FinancialTimesData
from kruppe.data_source.news.newsapi import NewsAPIData

from kruppe.data_source.finance.yfin import YFinanceData

rag_query_engine = RagQuery(
    retriever = retriever,
    llm = OpenAILLM()
)

llm_query_engine = LLMQuery(
    llm = OpenAILLM()
)

news_hub = NewsHub(news_sources=[
    NewYorkTimesData(headers_path="../../.nyt-headers.json"),
    FinancialTimesData(headers_path="../../.ft-headers.json"),
    NewsAPIData()
])

fin_hub = FinHub(
    fin_source = YFinanceData(),
    llm = OpenAILLM()
)

In [3]:
from kruppe.algorithm.librarian import Librarian

librarian = Librarian(
    llm=OpenAILLM(model="gpt-4.1"),
    docstore=docstore,
    index=index,
    toolkit = [
        rag_query_engine.rag_query,
        llm_query_engine.llm_query,
        news_hub.news_search,
        news_hub.news_recent,
        # news_hub.news_archive,
        fin_hub.get_company_background,
        fin_hub.get_company_income_stmt,
        fin_hub.get_company_balance_sheet,
        fin_hub.analyze_company_financial_stmts
    ]
)



In [4]:
query = "Who made more in revenue last year, NVIDIA or Tesla?"

In [5]:
librarian._messages = [
    {"role": "system", "content": librarian._react_system_prompt()},
    {"role": "user", "content": librarian._react_user_prompt(query)}
]

reason_messages, action = await librarian.reason(1)
print(reason_messages)
print(action)

[{'role': 'assistant', 'content': 'Thought 1: The information request is asking which company—NVIDIA or Tesla—had higher revenue in the most recent year ("last year"). To answer this, I need to retrieve the latest annual revenue numbers for both companies and compare them.\n\nPlan:\n1. Identify the ticker symbols for NVIDIA (NVDA) and Tesla (TSLA).\n2. Retrieve the most recent annual revenue for both companies using their income statements.\n3. Compare the revenue figures and identify which company had more revenue last year.\nAction 1: Request the most recent (last year) income statements for both NVIDIA (NVDA) and Tesla (TSLA) to obtain their annual revenue.\n'}]
Request the most recent (last year) income statements for both NVIDIA (NVDA) and Tesla (TSLA) to obtain their annual revenue.


In [6]:
librarian._messages.extend(reason_messages)
tool_call_messages, obs, sources = await librarian.act(1)
print(tool_call_messages)
print(obs)
print(sources)

[{'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_y24eLXLGnlrJ1DpnoPR5NtWv', 'type': 'function', 'function': {'name': 'get_company_income_stmt', 'arguments': '{"ticker": "NVDA", "years": 1}'}}]}, {'role': 'tool', 'tool_call_id': 'call_y24eLXLGnlrJ1DpnoPR5NtWv', 'content': 'Observation 1:                                                    2025-01-31     2024-01-31 Operating Revenue                              130497000000.0  60922000000.0 Total Revenue                                  130497000000.0  60922000000.0 Cost Of Revenue                                 32639000000.0  16621000000.0 Gross Profit                                    97858000000.0  44301000000.0 Selling General And Administration               3491000000.0   2654000000.0 Research And Development                        12914000000.0   8675000000.0 Operating Expense                               16405000000.0  11329000000.0 Operating Income                                81453000000.0  32972000000.0 

In [5]:
result = await librarian.execute(query, to_print=True)
result

Thought 1: The information request is for a comparison of last year's revenue between NVIDIA and Tesla. To answer this, I need to retrieve the most recent annual revenue figures for both companies. This information can be found in the most recent income statements for each company.

Plan:
1. Obtain the most recent (last year) income statement for NVIDIA (ticker: NVDA).
2. Obtain the most recent (last year) income statement for Tesla (ticker: TSLA).
3. Compare the "Revenue" or "Total Revenue" figures from both statements.
Action 1: Retrieve the most recent (last year) income statements for NVIDIA and Tesla.
get_company_income_stmt({"ticker": "NVDA", "years": 1})
Observation 1: 
                                                   2025-01-31     2024-01-31
Operating Revenue                              130497000000.0  60922000000.0
Total Revenue                                  130497000000.0  60922000000.0
Cost Of Revenue                                 32639000000.0  16621000000.0
Gross 

Response(text="for the most recent fiscal year, tesla made more in revenue than nvidia. \n- tesla's revenue (fiscal year ended dec 31, 2023): $96.8 billion.\n- nvidia's revenue (fiscal year ended jan 31, 2024): $60.9 billion.", sources=[])

In [4]:
query = "I want a preliminary background report on Tesla, including its recent news and financials."
response = await librarian.execute(query, to_print=True)

Thought 1: The information request asks for a preliminary background report on Tesla, specifically mentioning recent news and financials. To fulfill this, I should gather:
1. Company background information.
2. Recent news articles for current developments.
3. Financial statements (income statement and balance sheet or a summary)
4. A preliminary financial analysis based on recent data.

Since I need multiple types of information, I should perform several actions to collect them. The news search can collect recent updates; company background and financial statements can be retrieved by calling their respective functions. To enable efficient processing, I will use the multi_tool_use.parallel function.
Action 1: multi_tool_use.parallel({
  tool_uses: [
    {
      recipient_name: "functions.get_company_background",
      parameters: { ticker: "TSLA" }
    },
    {
      recipient_name: "functions.news_search",
      parameters: { days: 30, max_results: 10 }
    },
    {
      recipient_na

TIMEOUT ERROR ConnectTimeout: ConnectTimeout('')
[httpx] Failed to scrape https://finance.yahoo.com/news/csx-profit-drops-27-railroad-203559130.html : ConnectTimeout('')
TIMEOUT ERROR ConnectTimeout: ConnectTimeout('')
[httpx] Failed to scrape https://www.theverge.com/news/650285/google-gemini-live-screensharing-camera-feature-android-free : ConnectTimeout('')
TIMEOUT ERROR ConnectTimeout: ConnectTimeout('')
[httpx] Failed to scrape https://techcrunch.com/2025/04/16/zuckerberg-snapchat-would-have-grown-faster-if-it-accepted-6b-buyout-offer/ : ConnectTimeout('')
[Selenium] Failed to scrape https://techcrunch.com/2025/04/16/zuckerberg-snapchat-would-have-grown-faster-if-it-accepted-6b-buyout-offer/
TIMEOUT ERROR ConnectTimeout: ConnectTimeout('')
[httpx] Failed to scrape https://www.nascar.com/gallery/nascarcasm-most-memeable-moments-from-nascar-full-speed-season-1/ : ConnectTimeout('')
TIMEOUT ERROR ConnectTimeout: ConnectTimeout('')
[httpx] Failed to scrape https://www.axios.com/2025/0

Added 27 documents to index and docstore (out of 28 total documents)
news_search({"days": 30, "max_results": 10})
Observation 2:                                                 title  \
0   Gemini Live’s screensharing feature is now fre...   
1   New Xbox Doom Controller Features Spikes And B...   
2          Android 16 Beta 4 update set for tomorrow    
3   S&P 500 ekes out a gain, Dow tumbles 500 point...   
4   Chinese ‘teaspresso’ chain Chagee jumps on Wal...   
5   CSX profit drops 27% as railroad shipments dec...   
6                                                       
7                                                       
8   Trump administration halts Equinor’s $5bn New ...   
9   School Vouchers Won in Texas. Next Up, the Nat...   
10  Senator Lisa Murkowski Makes Startling Admissi...   
11  Former Times Editor Testifies in Sarah Palin D...   
12  Costa Rica Violated Rights of 81 Migrant Child...   
13  What to Know About the Court Cases Challenging...   
14  Elaine Wynn,

In [8]:
print(response.text)
print('='*80)
for doc in response.sources:
    print(doc.text)
    print(doc.metadata)
    print("-" * 80)

preliminary background report on tesla, inc.

1. company overview
tesla, inc. (tsla) designs, develops, manufactures, leases, and sells electric vehicles, as well as energy generation and storage systems in the united states, china, and internationally. founded in 2003 and headquartered in austin, texas, tesla operates through two main segments: automotive (electric vehicles and related services) and energy generation and storage (solar energy and battery products). as of the latest data, tesla employs over 125,000 people globally.

2. recent news
a targeted search for recent news about tesla over the past 30 days did not return any relevant articles in the available news databases. this may be due to limited coverage or the scope of the datasets searched.

3. financial overview & analysis (past 3 years)
- tesla saw modest revenue growth in 2024, but profitability margins contracted amid rising operating expenses and a significant tax charge.
- the company maintains a strong liquidity 